#### 250221 FlowKitを用いたFACS解析の流れ
- PythonでFCSファイルを加工するライブラリが存在するらしい。
- 実際に我々が取得したデータを用いてその出力を確かめる。

■ 参考
- https://flowkit.readthedocs.io/en/latest/index.html
- https://github.com/whitews/FlowKit

In [1]:
# ライブラリなどの準備

import bokeh
from bokeh.plotting import show
bokeh.io.output_notebook()

import os
import flowkit as fk
from flowkit._utils import plot_utils

os.chdir('C:/github/omics-util/fcs_analysis')  # カレントディレクトリの移動

Loading BokehJS ...

***
#### 1. データの読み込み

In [2]:
sample = fk.Sample("./data/Specimen_001_APAP_Low_1_003.fcs")
display(sample.channels)

,channel_number,pnn,pns,png,pne,pnr
0,1,FSC-A,,1.00,"(0.0, 0.0)",262144.0
1,2,FSC-H,,1.00,"(0.0, 0.0)",262144.0
2,3,FSC-W,,1.00,"(0.0, 0.0)",262144.0
3,4,SSC-A,,1.00,"(0.0, 0.0)",262144.0
4,5,SSC-H,,1.00,"(0.0, 0.0)",262144.0
5,6,SSC-W,,1.00,"(0.0, 0.0)",262144.0
6,7,PE-A,CD45,1.00,"(0.0, 0.0)",262144.0
7,8,PE-H,CD45,1.00,"(0.0, 0.0)",262144.0
8,9,7-AAD-A,Dump,1.00,"(0.0, 0.0)",262144.0
9,10,7-AAD-H,Dump,1.00,"(0.0, 0.0)",262144.0


#### 2. 分布の確認

In [3]:
# ヒストグラムとして可視化
p = sample.plot_histogram('FSC-H', source='raw', bins=256)
show(p)

# 散布図として可視化
f = sample.plot_contour('FSC-H', 'SSC-A', source='raw', plot_events=True)
show(f)

In [4]:
# スケールの変更
asinh_xform = fk.transforms.AsinhTransform(
    param_t=262144,
    param_m=1,
    param_a=0.0
)
sample.apply_transform(asinh_xform, include_scatter=True)  # オプション追加

# スケール変換後に描画
p = sample.plot_scatter('FSC-H', 'SSC-A', source='xform', subsample=True)
show(p)



- FSCとSSCのように、蛍光チャネル以外の軸に対しても変換を適用するには`include_scatter=True`を追記する必要がある点に注意。
- issueでも指摘されていた。[Issue #51](https://github.com/whitews/FlowKit/issues/54)

***
#### 3. ゲーティングの実施
1. スケール変換
2. GatingStrategyインスタンスにスケール変換を追加（後で呼び出すため）
3. 各種ゲーティングの実施
4. 親を指定して下流に伸ばしていく
5. 可視化して確認

In [5]:
g_strat = fk.GatingStrategy()  # Create a new GatingStrategy instance

#### 1. FSC-H vs FSC-A ####
asinh_xform = fk.transforms.AsinhTransform(
    param_t=262144,
    param_m=1,
    param_a=0.0
)
sample.apply_transform(asinh_xform, include_scatter=True)

# 可視化してポリゴンを決定
p = sample.plot_scatter('FSC-H', 'SSC-A', source='xform', subsample=True)
show(p)


g_strat.add_transform('asinh1', asinh_xform)

dim_a = fk.Dimension('FSC-H', range_min=0.0, range_max=2, transformation_ref="asinh1")
dim_b = fk.Dimension('SSC-A', range_min=0.0, range_max=2, transformation_ref="asinh1")

# polygon gating
vertices = [
    (0.2, 2e-2),
    (0.5, 2e-2),
    (0.7, 0.2),
    (0.9, 0.6),
    (0.9, 1.5),
    (0.4, 1.5),
    (0.2, 0.3)
]

poly_gate = fk.gates.PolygonGate(
    'fsc_ssc',
    dimensions=[dim_a, dim_b],
    vertices=vertices
)
g_strat.add_gate(poly_gate, gate_path=('root',))
res = g_strat.gate_sample(sample)

`sample.plot_scatter()`で実際に分布を見ながらポリゴンの座標を決定する。少し手間。

In [6]:
#### 2. FSC-H vs FSC-A ####
dim_a = fk.Dimension('FSC-H', range_max=1, transformation_ref="asinh1")  # 上で定義した変換を使いまわす
dim_b = fk.Dimension('FSC-A', range_max=1, transformation_ref="asinh1")

# polygon gating
vertices = [
    (0.05, 0),
    (0.1, 0),
    (0.9, 1.0),
    (0.9, 1.2),
    (0.05, 0.2),
]

poly_gate = fk.gates.PolygonGate(
    'fsc_fsc',
    dimensions=[dim_a, dim_b],
    vertices=vertices
)
g_strat.add_gate(poly_gate, gate_path=('root','fsc_ssc'))
res = g_strat.gate_sample(sample)

In [7]:
#### 3. CD45 vs Dump ####
asinh_xform2 = fk.transforms.AsinhTransform(
    param_t=262144,
    param_m=4.0,
    param_a=0.0
)
sample.apply_transform(asinh_xform2, include_scatter=True)
g_strat.add_transform('asinh2', asinh_xform2)

dim_a = fk.Dimension('PE-A', range_max=1, transformation_ref="asinh2")
dim_b = fk.Dimension('7-AAD-A', range_max=1, transformation_ref="asinh2")

# polygon gating
vertices = [
    (0.6, 0.5),
    (0.82, 0.5),
    (0.82, 0.75),
    (0.6, 0.75),
]

poly_gate = fk.gates.PolygonGate(
    'cd45_dump',
    dimensions=[dim_a, dim_b],
    vertices=vertices
)
g_strat.add_gate(poly_gate, gate_path=('root','fsc_ssc','fsc_fsc'))
res = g_strat.gate_sample(sample)

軸の組み合わせによってスケーリングのパラメータは変わる。新規に作成した場合は別名で保存する。

In [8]:
#### 4. CD3 vs NK
asinh_xform3 = fk.transforms.AsinhTransform(
    param_t=262144,
    param_m=3.3,
    param_a=0.0
)
sample.apply_transform(asinh_xform3, include_scatter=True)
p = sample.plot_scatter('FITC-A', 'APC-A', source='xform', subsample=True)
show(p)

g_strat.add_transform('asinh3', asinh_xform3)


quad_div1 = fk.QuadrantDivider(
    'cd3-div',
    'FITC-A',
    compensation_ref='uncompensated',
    transformation_ref="asinh3",
    values=[0.4]
)
quad_div2 = fk.QuadrantDivider(
    'nk-div',
    'APC-A',
    compensation_ref='uncompensated',
    transformation_ref="asinh3",
    values=[0.2]
)
quad_divs = [quad_div1, quad_div2]

# the 2 dividers above will be used to divide the space into 4 quadrants
quad_1 = fk.gates.Quadrant(
    quadrant_id='CD3pos-NKpos',
    divider_refs=['cd3-div', 'nk-div'],
    divider_ranges=[(0.4, None), (0.2, None)]
)
quad_2 = fk.gates.Quadrant(
    quadrant_id='CD3pos-NKneg',
    divider_refs=['cd3-div', 'nk-div'],
    divider_ranges=[(0.4, None), (None, 0.2)]
)
quad_3 = fk.gates.Quadrant(
    quadrant_id='CD3neg-NKpos',
    divider_refs=['cd3-div', 'nk-div'],
    divider_ranges=[(None, 0.4), (0.2, None)]
)
quad_4 = fk.gates.Quadrant(
    quadrant_id='CD3neg-NKneg',
    divider_refs=['cd3-div', 'nk-div'],
    divider_ranges=[(None, 0.4), (None, 0.2)]
)
quadrants = [quad_1, quad_2, quad_3, quad_4]

# We can now construct our QuadrantGate
quad_gate1 = fk.gates.QuadrantGate(
    'quadgate1',
    dividers=quad_divs,
    quadrants=quadrants
)

g_strat.add_gate(quad_gate1, gate_path=('root','fsc_ssc','fsc_fsc','cd45_dump'))
res = g_strat.gate_sample(sample)


ポリゴン以外にも、定めた座標からの四分位範囲をゲーティングする設定もある。使いそう。

***
##### 一連のゲーティング結果を可視化する


In [9]:
# boolean gate
gate_refs = [
    {
        'ref': 'fsc_ssc',
        'path': ('root',),
        'complement': False
    },
    {
        'ref': 'fsc_fsc',
        'path': ('root', 'fsc_ssc'),
        'complement': False
    },
    {
        'ref': 'cd45_dump',
        'path': ('root', 'fsc_ssc', 'fsc_fsc'),
        'complement': False
    },
    
    {
        'ref': 'CD3pos-NKpos',
        'path': ('root', 'fsc_ssc', 'fsc_fsc', 'cd45_dump'),
        'complement': False
    },
]
# 1. FSC-H vs FSC-A
res = g_strat.gate_sample(sample)
res.report

tmp_gate = res.get_gate_membership('fsc_ssc')
asinh_xform = fk.transforms.AsinhTransform(
    param_t=262144,
    param_m=1,
    param_a=0.0
)
sample.apply_transform(asinh_xform, include_scatter=True)
p = sample.plot_scatter('FSC-H', 'SSC-A', source='xform', highlight_mask=tmp_gate)
show(p)

# 2. FSC-H vs FSC-A
tmp_gate = res.get_gate_membership('fsc_fsc')
asinh_xform = fk.transforms.AsinhTransform(
    param_t=262144,
    param_m=1,
    param_a=0.0
)
sample.apply_transform(asinh_xform, include_scatter=True)
p = sample.plot_scatter('FSC-H', 'FSC-A', source='xform', highlight_mask=tmp_gate)
show(p)

# 3. CD45 vs Dump
tmp_gate = res.get_gate_membership('cd45_dump')
asinh_xform2 = fk.transforms.AsinhTransform(
    param_t=262144,
    param_m=4.0,
    param_a=0.0
)
sample.apply_transform(asinh_xform2, include_scatter=True)
p = sample.plot_scatter('PE-A', '7-AAD-A', source='xform', highlight_mask=tmp_gate)
show(p)

# 4. CD3 vs NK
tmp_gate = res.get_gate_membership('CD3pos-NKpos')
asinh_xform3 = fk.transforms.AsinhTransform(
    param_t=262144,
    param_m=3.3,
    param_a=0.0
)
sample.apply_transform(asinh_xform3, include_scatter=True)
p = sample.plot_scatter('FITC-A', 'APC-A', source='xform', highlight_mask=tmp_gate)
show(p)


In [10]:
# 最終的なゲート構造を出力
print(g_strat.get_gate_hierarchy(output='ascii'))

root
╰── fsc_ssc
    ╰── fsc_fsc
        ╰── cd45_dump
            ╰── quadgate1
                ├── CD3pos-NKpos
                ├── CD3pos-NKneg
                ├── CD3neg-NKpos
                ╰── CD3neg-NKneg


***
#### まとめ・感想
- FlowKitを用いてゲーティングまで行えることを確認した。
- FCSファイルを1から加工するとなると***不便さ***が勝る。
   - ゲーティングが直観的でない。UI上は測定点の座標が読めるだけで、手動でくくることはできない。ポリゴンの座標を毎回打つのが大変。
   - ゲーティングした後の可視化も困難。ゲーティングされた箇所以外の透明度が高くなる描画方針であり、輪郭が表示されない。おそらくFlowJoのワークスペースファイルを読み込む設定だと表示できそう。
- FlowJoの測定結果を定量するライブラリとしては使えそうだが、それくらいなら作れる。